In [217]:
from keras.utils import Sequence
from keras.utils import to_categorical
import numpy as np
import librosa
import os
import IPython.display as ipd

In [255]:
class Chordgen(Sequence):
    def __init__(self, data, max_strings, n, batch_size, samples):
        self.data = data.reset_index(drop=True)
        self.max_strings = max_strings
        self.n = n
        self.batch_size = batch_size
        self.samples = samples
        self.on_epoch_end()
        
    def on_epoch_end(self):
        self.indexes = np.arange(self.data.shape[0])
        np.random.shuffle(self.indexes)
        
    def __data_generation(self, selected):
        data = self.data.loc[np.array(selected)].copy()
        X = np.zeros((0, 4000//self.samples, self.samples))
        y = np.zeros((0, 6))
        
        for index, row in data.iterrows():
            for _ in range(self.n):
                n_strings = np.random.choice(np.arange(1, self.max_strings + 1))
                x, frets = self.create_chord(row, n_strings)
                x = self.to_spectral(x, self.samples)
                X = np.vstack((X, x.reshape(1, x.shape[0], x.shape[1])))
                y = np.vstack((y, frets))
                
        return X, to_categorical(y)
    
    def __len__(self):
        "Batches per epoch"
        return int(np.floor(self.data.shape[0] / self.batch_size))
    
    def __getitem__(self, index):
        selected = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X, y = self.__data_generation(selected)
        
        return X, y
            
    def create_chord(self, note, n_strings):
        n = 1
        note_frets = note["Notes"]
        note_x = note["x"]
        note_string = int(note["String"])

        # Add initial note to chord
        x = np.pad(note_x, (0, 4000 - note_x.shape[0]), mode="constant")
        frets = note_frets.copy()

        while n < n_strings:
            # Select next note
            next_note = self.data.sample()
            next_note_frets = next_note["Notes"].values[0].copy()
            next_note_x = next_note["x"].values[0]
            next_note_string = int(next_note["String"].values[0])

            # Check it's on a different string and the frets are close
            next_fret = next_note_frets[next_note_string - 1].copy()
            cond1 = frets[next_note_string - 1] == 25 # No sound in next note's string
            cond2 = np.max(np.abs(frets[frets != 25] - next_fret)) <= 6 # 6 or less frets distance
            if cond1 and cond2:
                frets[next_note_string - 1] = next_fret.copy()
                x += np.pad(next_note_x, (0, 4000 - next_note_x.shape[0]), mode="constant")
                n += 1
                
        # Modulate wave's amplitude
        x = np.random.normal(0.7, 0.2) * x
        
        return x, frets
    
    def to_spectral(self, x, samples):
        xf = np.zeros((x.shape[0] // samples, samples))
        for i in range(0, x.shape[0] - samples, samples):
            w = abs(np.fft.fft(x[i:i+samples], n=samples*2))
            freqs = np.fft.fftfreq(len(w))
            xf[i//samples, :] = w[freqs >= 0]
        return xf